<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_production_prototype_heft8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creariamo un ambiente TEST

In [1]:
!mkdir -p /content/calcio

In [3]:
file_quantity = 200
!wget -q -O /content/calcio/541001.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/nsvzD9fEWvO8jw
for num in range(file_quantity):
    !cp /content/calcio/541001.csv /content/calcio/{541002 + num * 2 + 1}.csv

### Project config

In [4]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw
     

/usr/local/lib/python3.8/dist-packages/neptune/internal/backends/hosted_client.py:48: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-4-2aa8b85c691b>:5: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [5]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [6]:
!pip install jupyter-dash >> None
!pip install dash-bootstrap-components >> None
!pip install catboost >> None

### Imports

In [7]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.3.5
1.22.4


In [8]:
from glob import glob
import yaml
from os.path import isfile
from os.path import isdir
import subprocess
from time import gmtime, strftime, time

In [9]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash import dash_table

In [10]:
from catboost import CatBoost
from scipy.stats import poisson

In [11]:
INPUT_DIR = '/content/calcio/'
#1 X 2 T0.5 T1 T1.5 T2 H-2 H-1.5 H-1 H-0.5 H+0 H+0.5 H1 H+1.5 H+2

### Code

#### Input Constants

In [12]:
data_types_dict = {
    'Id':np.int32, 
    'StatTime':np.datetime64, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64, 
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [13]:
cols = ['StatTime', 'Minute', 'Active', 'Score1', 'Score2', 'A1', 'A2', 'DA1',
       'DA2', 'Pos1', 'Pos2', 'Off1', 'Off2', 'On1', 'On2', 'YC1', 'YC2',
       'RC1', 'RC2', 'Sub1', 'Sub2', 'Pen1', 'Pen2', 'Cor1', 'Cor2', 'Period',
       'Comment']

In [14]:
usecols = ['Minute', 'Active', 'Score1', 'Score2', 'A1', 'A2', 'DA1',
       'DA2', 'Pos1', 'Pos2', 'Off1', 'Off2', 'On1', 'On2', 'YC1', 'YC2',
       'RC1', 'RC2', 'Sub1', 'Sub2', 'Pen1', 'Pen2', 'Cor1', 'Cor2']

In [15]:
match_cols = ['min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'A1relativ', 'A2relativ', 'DA1_scaled', 'DA2_scaled',
       'DA1perMIN', 'DA2perMIN', 'DA1relativ', 'DA2relativ', 'Pos1_cleaned',
       'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm', 'On2_norm',
       'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed']

#### Functions

In [16]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

In [17]:
def update_models_dict(add_dict:dict):
    if isfile('/content/load_models.yaml'):
        with open('/content/load_models.yaml', "r") as yml:
            current_yaml = yaml.load(yml, Loader=yaml.SafeLoader)
        for key, value in add_dict.items():
            current_yaml[key] = value        
        with open('/content/load_models.yaml', "w") as yml:
            yaml.dump(current_yaml, yml)
    else:
        with open('/content/load_models.yaml', "w") as yml:
            yaml.dump(current_yaml, yml)

In [18]:
def load_model(model_type_order:str, model_name:str, num:str, api_key):
    if not isdir(f"{INPUT_DIR}models"):
        bashCommand = f"""
        mkdir -p {INPUT_DIR}models
                        """
        run_bash(bashCommand, 'tar_model')
    model_type, order = model_type_order.split(':')
    model_num = str(num)
    model_version_params = dict(
        project = 'scomesse/football',
        model = model_name,
        api_token = api_key,
        with_id = model_name + '-' + model_num
    )
    PATH_TO_MODEL = f"{INPUT_DIR}models/booster_{model_type}_{model_name}_{model_num}.model"
    if isfile('./log_load_models.txt'):
        with open('./log_load_models.txt', 'a') as f:
            f.write(model_name + '-' + model_num + '\n')
            f.write(PATH_TO_MODEL + "\n")
    else:
        with open('./log_load_models.txt', 'w') as f:
            f.write(model_name + '-' + model_num + '\n')
            f.write(PATH_TO_MODEL + "\n")

    model_version = neptune.init_model_version(**model_version_params)
    model_version['model'].download(PATH_TO_MODEL)
    model_version.stop()
    if model_type in model_dict:
        model_dict[model_type].update({order:CatBoost()})
    else:
        model_dict[model_type] = {order:CatBoost()}
    model_dict[model_type][order].load_model(PATH_TO_MODEL)
    return PATH_TO_MODEL
    #{model_type: {f'model{order}_path': PATH_TO_MODEL}}
    #update_models_dict(add_dict:dict)

In [19]:
def create_predict_vector(file_path:str):
    match_df = pd.read_csv(
                        file_path, 
                        sep = ';', 
                        names = cols, 
                        skiprows = 1, 
                        usecols = usecols,
                        dtype = data_types_dict
                            )
    match_df.iloc[0, :] = match_df.iloc[0, :].fillna(0)
    match_df = match_df.fillna(method = 'ffill')
    P1, PX, P2 = pd.read_csv(
                        file_path, 
                        sep = ';', 
                        nrows = 0,
                            )
    match_df.loc[:,['P1', 'P2']] = 1 / np.array((P1, P2), dtype = np.float32)
    match_df['min_norm'] = match_df['Minute'].astype(np.float32) / 50
    # трансформируем голы
    match_df[match_df['Score1'].isna()] = 0
    match_df['Score1_norm'] = match_df['Score1'].fillna(method = 'ffill').fillna(0).astype(np.float32) / 4
    match_df.loc[match_df['Score1'] > 3, ['Score1_norm']] = 1.0
    match_df[match_df['Score2'].isna()] = 0
    match_df['Score2_norm'] = match_df['Score2'].fillna(method = 'ffill').fillna(0).astype(np.float32) / 4
    match_df.loc[match_df['Score2'] > 3, ['Score2_norm']] = 1.0
    match_df['Score_diff'] = match_df['Score1'].astype(np.int16) - match_df['Score2'].astype(np.int16)
    match_df.loc[match_df['Score_diff'] < -4, ['Score_diff']] = -4
    match_df.loc[match_df['Score_diff'] > 4, ['Score_diff']] = 4
    match_df[[f'Score_cat_{n}' for n in range(1, 10)]] = np.eye(9)[match_df['Score_diff'].values]
    match_df['Score_diff'] = match_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    #трансформируем атаки
    match_df['A1_scaled'] = match_df['A1'].astype(np.float32) / 75
    match_df.loc[match_df['A1'] >= 60, ['A1_scaled']] = (60 + (match_df['A1'] - 60) / 4) / 75
    match_df['A2_scaled'] = match_df['A2'].astype(np.float32) / 75
    match_df.loc[match_df['A2'] >= 60, ['A2_scaled']] = (60 + (match_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    match_df['A1perMIN'] = match_df['A1'].astype(np.float32) / match_df['Minute'].astype(np.float32)
    match_df.loc[match_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    match_df['A2perMIN'] = match_df['A2'].astype(np.float32) / match_df['Minute'].astype(np.float32)
    match_df.loc[match_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    match_df['A1relativ'] = (match_df['A1'].astype(np.float32) - match_df['A1'].shift(5).astype(np.float32)).fillna(0)
    match_df.loc[match_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    match_df['A2relativ'] = (match_df['A2'].astype(np.float32) - match_df['A2'].shift(5).astype(np.float32)).fillna(0)
    match_df.loc[match_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    # трансформируем опасные атаки
    match_df['DA1_scaled'] = match_df['DA1'].astype(np.float32) / 50
    match_df.loc[match_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (match_df['DA1'] - 40) / 3) / 100
    match_df['DA2_scaled'] = match_df['DA2'].astype(np.float32) / 50
    match_df.loc[match_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (match_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    match_df['DA1perMIN'] = match_df['DA1'].astype(np.float32) / match_df['Minute'].astype(np.float32)
    match_df.loc[match_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    match_df['DA2perMIN'] = match_df['DA2'].astype(np.float32) / match_df['Minute'].astype(np.float32)
    match_df.loc[match_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    match_df['DA1relativ'] = (match_df['DA1'].astype(np.float32) - match_df['DA1'].shift(5).astype(np.float32)).fillna(0)
    match_df.loc[match_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    match_df['DA2relativ'] = (match_df['DA2'].astype(np.float32) - match_df['DA2'].shift(5).astype(np.float32)).fillna(0)
    match_df.loc[match_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    # Владение мячом
    match_df['Pos1_cleaned'] = match_df['Pos1'].fillna(method = 'ffill').fillna(0).astype(np.float32) /  np.float32(100.0)
    match_df.loc[match_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    match_df.loc[match_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    match_df['Pos2_cleaned'] = match_df['Pos2'].fillna(method = 'ffill').fillna(0).astype(np.float32) /  np.float32(100.0)
    match_df.loc[match_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    match_df.loc[match_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    # трансформируем удары
    match_df['Off1_norm'] = match_df['Off1'].fillna(method = 'ffill').fillna(0).astype(np.float32) / np.float32(10.0)
    match_df.loc[match_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    match_df['Off2_norm'] = match_df['Off2'].fillna(method = 'ffill').fillna(0).astype(np.float32) / np.float32(10.0)
    match_df.loc[match_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    # трансформируем удары в створ
    match_df['On1_norm'] = match_df['On1'].fillna(method = 'ffill').fillna(0).astype(np.float32) / np.float32(5.0)
    match_df.loc[match_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    match_df['On2_norm'] = match_df['On2'].fillna(method = 'ffill').fillna(0).astype(np.float32) / np.float32(5.0)
    match_df.loc[match_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    # Желтые карточки
    match_df['YC1_transformed'] = match_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    match_df.loc[match_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    match_df['YC2_transformed'] = match_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    match_df.loc[match_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    # трансформируем красные карточки
    match_df['RC1_transformed'] = match_df['RC1'].fillna(0).astype(np.int8)
    match_df.loc[match_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    match_df['RC2_transformed'] = match_df['RC2'].fillna(0).astype(np.int8)
    match_df.loc[match_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    # Замены
    match_df['Sub1_transformed'] = match_df['Sub1'].fillna(0).astype(np.int8)
    match_df.loc[match_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    match_df['Sub2_transformed'] = match_df['Sub2'].fillna(0).astype(np.int8)
    match_df.loc[match_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    # Угловые
    match_df['Cor1_transformed'] = match_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    match_df.loc[match_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    match_df['Cor2_transformed'] = match_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    match_df.loc[match_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    # Кэфы
    match_df['P1_transformed'] = np.log(match_df['P1'], dtype = np.float32) / 2
    match_df['P2_transformed'] = np.log(match_df['P2'], dtype = np.float32) / 2
    return match_df[match_cols].values[-1,:]

In [20]:
def total_probability(regression_vector1, regression_vector2):
    poisson_dict = {}
    poisson_dict[1] = {}
    poisson_dict[2] = {}
    for goal in range(7):
        poisson_dict[1][goal] = poisson.pmf(goal, regression_vector1)
        poisson_dict[2][goal] = poisson.pmf(goal, regression_vector2)

    # Считаем вероятности суммы забитых мячей
    total_matrix = np.zeros(13)
    for goal1 in range(7):
        for goal2 in range(7):
            total_matrix[goal1 + goal2] = total_matrix[goal1 + goal2] + \
            poisson_dict[1][goal2] * poisson_dict[2][goal1]
    
    # Считаем вероятности забить не менее определенного количества мячей
    over_matrix = np.flip(np.cumsum(np.flip(total_matrix)))    
    # Считаем вероятности забить не более определенного количества
    under_matrix = np.cumsum(total_matrix)
    return dict(
            under_05 = under_matrix[0], over_05 = over_matrix[1],
            under_10 = under_matrix[0] / (under_matrix[0] + over_matrix[2]), 
            over_10 = over_matrix[2] / (under_matrix[0] + over_matrix[2]),
            under_15 = under_matrix[1], over_15 = over_matrix[2],
            under_20 = under_matrix[1] / (under_matrix[1] + over_matrix[3]), 
            over_20 = over_matrix[3] / (under_matrix[1] + over_matrix[3])
                )

In [21]:
def handicap_probability(regression_vector1, regression_vector2):
    poisson_dict = {}
    poisson_dict[1] = {}
    poisson_dict[2] = {}
    for goal in range(7):
        poisson_dict[1][goal] = poisson.pmf(goal, regression_vector1)
        poisson_dict[2][goal] = poisson.pmf(goal, regression_vector2)

    # Считаем вероятности суммы забитых мячей
    total_matrix = np.zeros(13)
    for goal1 in range(7):
        for goal2 in range(7):
            total_matrix[goal1 - goal2 + 6] = total_matrix[goal1 - goal2 + 6] + \
            poisson_dict[1][goal2] * poisson_dict[2][goal1]

    # Считаем вероятности победы дома over = home_win
    hcap_home = np.cumsum(np.flip(total_matrix)) 
    # Считаем вероятности победы гостей under = away_win
    hcap_away = np.cumsum(total_matrix)

    return dict(
            home_m20_win = hcap_home[3] / (hcap_home[3] + hcap_away[7]),
            home_m20_lose = hcap_away[7] / (hcap_home[3] + hcap_away[7]),
            home_m15_win = hcap_home[4],
            home_m15_lose = hcap_away[7],
            home_m10_win = hcap_home[4] / (hcap_home[4] + hcap_away[6]),
            home_m10_lose = hcap_away[6] / (hcap_home[4] + hcap_away[6]),
            home_m05_win = hcap_home[5],
            home_m05_lose = hcap_away[6],
            home_m00_win = hcap_home[5] / (hcap_home[5] + hcap_away[5]),
            home_m00_lose = hcap_away[5] / (hcap_home[5] + hcap_away[5]),
            home_p05_win = hcap_home[6],
            home_p05_lose = hcap_away[5],
            home_p10_win = hcap_home[6] / (hcap_home[6] + hcap_away[4]),
            home_p10_lose = hcap_away[4] / (hcap_home[6] + hcap_away[4]),
            home_p15_win = hcap_home[7],
            home_p15_lose = hcap_away[4],
            home_p20_win = hcap_home[7] / (hcap_home[7] + hcap_away[3]),
            home_p20_lose = hcap_away[3] / (hcap_home[7] + hcap_away[3])
                    )

#### run DASH server

In [22]:
model_path_dict ={}
model_dict = {}
app = JupyterDash(__name__,
                  external_stylesheets = [dbc.themes.FLATLY])

In [23]:
# TABS CONTENT
tab1_content = [dbc.Row([
                        dbc.Col(
                            dbc.Button('predict', id = 'predict_button', 
                                    n_clicks = 0, className = 'mr-2')
                            ),
                        dbc.Col(
                            html.Div('Запустить предикт')
                            )                                
                        ], style = {'margin-top':20, 'margin-bottom':40}),
                dbc.Row([
                        dbc.Col(html.Div(id = 'predict-info-1')),
                        dbc.Col(html.Div(id = 'predict-info-2'))
                        ], style = {'margin-bottom':40})]
tab2_content = [dbc.Row(html.Div(id = 'data-table'), style = {'margin-top':40})]

In [24]:
# TABS CONTENT 3
tab3_content = [
######################## выбор 1x2 ##############################
    dbc.Row(html.Div([
        html.Div('Тип ставки: 1Х2 | Модель | Версия'),
        dcc.Input(id='model-1-name', type='text', value='FOOT-LIVEMC'),
        dcc.Input(id='model-1-num', type='text', value=2),
                        ], style = {'margin-top':5, 'margin-bottom':5})
            ),
######################## выбор handicap ##########################
    dbc.Row(html.Div([
        html.Div('Тип ставки: HANDICAP | Модель n.1 | Версия'),
        dcc.Input(id='model-2-name', type='text', value='FOOT-LIVEBST1'),
        dcc.Input(id='model-2-num', type='text', value=3),
                        ], style = {'margin-top':5, 'margin-bottom':5})
            ),
    dbc.Row(html.Div([
        html.Div('Тип ставки: HANDICAP | Модель n.2 | Версия'),
        dcc.Input(id='model-3-name', type='text', value='FOOT-LIVEBST2'),
        dcc.Input(id='model-3-num', type='text', value=4),
                        ], style = {'margin-top':5, 'margin-bottom':5})
            ),
######################## выбор total #############################
    dbc.Row(html.Div([
        html.Div('Тип ставки: TOTAL | Модель n.1 | Версия'),
        dcc.Input(id='model-4-name', type='text', value='FOOT-LIVEBST1'),
        dcc.Input(id='model-4-num', type='text', value=3),
                        ], style = {'margin-top':5, 'margin-bottom':5})
            ),
    dbc.Row(html.Div([
        html.Div('Тип ставки: TOTAL | Модель n.2 | Версия'),
        dcc.Input(id='model-5-name', type='text', value='FOOT-LIVEBST2'),
        dcc.Input(id='model-5-num', type='text', value=4),
                        ], style = {'margin-top':5, 'margin-bottom':5})
            ),
######################## выбор total #############################
    dbc.Row([
            dbc.Col(
                dbc.Button('load', id = 'start_butt', 
                            n_clicks = 0, className = 'mr-2')
                    )             
            ], style = {'margin-top':20, 'margin-bottom':40}),
    dbc.Row([
            html.Div(id = 'output-info')
            ], style = {'margin-top':40})
                ]

In [25]:
app.layout = html.Div([
        #_header
    #dcc.Store(id='models_dict', storage_type='session', data = dict),
    dbc.Row(html.H4('Live Prediction Dashboard'),
            style = {'margin-bottom':40}),
    dbc.Tabs([
                dbc.Tab(tab1_content, label = 'Prediction'),
                dbc.Tab(tab2_content, label = 'Files'),
                dbc.Tab(tab3_content, label = 'Load Models')
                ]),
    # dcc.Store inside the user's current browser session
    dcc.Store(id='model-path-dict', storage_type='memory') # 'local' or 'session'
    ],
                    style = {'margin-left': '80px', 'margin-right': '80px'}
)

In [26]:
@app.callback(
    output= {
        'info':Output('output-info', 'children')
        },
    inputs = {'start_button':Input('start_butt', 'n_clicks')},
    state = {
        'names':{cnt:State(f'model-{cnt}-name', 'value') for cnt in range(1, 6)},
        'nums':{cnt:State(f'model-{cnt}-num', 'value') for cnt in range(1, 6)},
        }
              )
def update_click(start_button, names, nums):
    if start_button > 0:
        start_time = time()
        model_types = ['', '1x2:1', 'total:1', 'total:2', 'handicap:1', 'handicap:2']
        for nmod in range (1, 6):
            name = names[nmod]
            mnum = nums[nmod]
            model_type = model_types[nmod]
            model_path_dict[model_type] = load_model(model_type, name, mnum, api_key)
        return {
            'info':f'''
            Download models:
            |1x2|: {names[1]} - {nums[1]}
            ***|HANDICAP m.1|: {names[2]} - {nums[2]}
            ***|HANDICAP m.2|: {names[3]} - {nums[3]}
            ***|TOTAL m.1|: {names[4]} - {nums[4]}
            ***|TOTAL m.2|: {names[5]} - {nums[5]} in _{time() - start_time}_sec
                    '''
        }


In [27]:
@app.callback(
    output= {'info':Output('predict-info-1', 'children')},
    inputs = {'predict_button':Input('predict_button', 'n_clicks')}
              )
def update_click(predict_button):
    if predict_button > 0:
        start_time = time()
        output_dict = {}
        for file_path in glob(INPUT_DIR + '*.csv'):
            file_num = file_path.split('/')[-1].split('.')[0]
            output_dict[file_num] = {}
            input_vector = create_predict_vector(file_path)
            output_dict[file_num]['mc_home'], output_dict[file_num]['mc_draw'], output_dict[file_num]['mc_away'] =  \
                            (np.flip(model_dict['1x2']['1'].predict(input_vector, prediction_type="Probability")))
            output_dict[file_num].update(
                total_probability(
                    model_dict['total']['1'].predict(input_vector) * 21, 
                    model_dict['total']['2'].predict(input_vector) * 21
                                ))
            output_dict[file_num].update(
                handicap_probability(
                    model_dict['handicap']['1'].predict(input_vector) * 21, 
                    model_dict['handicap']['2'].predict(input_vector) * 21
                                ))
            predict_time = time()
            pd.DataFrame.from_dict(output_dict, orient = 'index').to_csv('./output.csv')
        return {
            'info':f'''
            {strftime("%Y-%m-%d %H:%M:%S", gmtime())}: predict ready in _{time() - start_time}_sec,
            ______predict time {predict_time - start_time}
                    '''
        }


#### run server

In [28]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>